In [1]:
import numpy as np

In [2]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
message = 'My name is Vuong Van Duy from group 1362'
#message = 'VUONGVANDUY'
# hill matrix  
# 2 15 22  3
#  1  9  1 12
# 16  7 13 11
#  8  5  9  6
Hill_matrix = np.array([[2, 15, 22, 3], [1, 9, 1, 12], [16, 7, 13, 11], [8, 5, 9, 6]])

In [3]:
# in ma trận hill ra file excel
import pandas as pd
df = pd.DataFrame(Hill_matrix)
df.to_excel('output/Hill_matrix.xlsx', index=False, header=False)

In [4]:
def get_plaintext_matrix(message):
    message = message.upper()
    message = message.replace(' ', '')
    # thay thế các kí tự k phải chữ cái bằng kí tự ''
    for i in message:
        if i not in alphabet:
            message = message.replace(i, '')
    plaintext_matrix = [alphabet.index(i) for i in message]
    if len(plaintext_matrix) % len(Hill_matrix) != 0:
        plaintext_matrix += [23] * (len(Hill_matrix) - len(plaintext_matrix) % len(Hill_matrix))
    plaintext_matrix = np.array(plaintext_matrix)
    plaintext_matrix = plaintext_matrix.reshape(-1, 4)
    
    return plaintext_matrix

In [5]:
plaintext_matrix = get_plaintext_matrix(message)
print(plaintext_matrix)

[[12 24 13  0]
 [12  4  8 18]
 [21 20 14 13]
 [ 6 21  0 13]
 [ 3 20 24  5]
 [17 14 12  6]
 [17 14 20 15]]


In [6]:
# in ma trận plaintext ra file excel xóa đi dòng thứ tự cột ở đầu
df = pd.DataFrame(plaintext_matrix)
df.to_excel('output/plaintext_matrix.xlsx', index=False, header=False)

In [7]:
def fix_format_ciphertext(message, ciphertext):
    list_chars = list(message)
    for i in range(len(list_chars)):
        if not list_chars[i].isalpha():
            ciphertext = ciphertext[:i] + list_chars[i] + ciphertext[i:]
            list_chars[i] = '-'
        if list_chars[i].islower():
            ciphertext = ciphertext[:i] + ciphertext[i].lower() + ciphertext[i+1:]
            
    return ciphertext

In [8]:
def encrypt_Hill(message, Hill_matrix, mode=False):
    plaintext_matrix = get_plaintext_matrix(message)
    cipher_matrix = np.dot(plaintext_matrix, Hill_matrix) % 26
    encrypted_message = cipher_matrix.flatten()
    encrypted_message = [alphabet[i] for i in encrypted_message]
    encrypted_message = ''.join(encrypted_message)
    if mode:
        encrypted_message = fix_format_ciphertext(message, encrypted_message)
    return encrypted_message, cipher_matrix

In [9]:
cipher_text, cipher_matrix = encrypt_Hill(message, Hill_matrix)
print(cipher_matrix)
print(cipher_text)

[[22 19 15 25]
 [14 24 14 20]
 [ 0  8  1 15]
 [ 7  6 10 10]
 [ 8  2  1 23]
 [ 2  1  0 23]
 [20 24  3  9]]
WTPZOYOUAIBPHGKKICBXCBAXUYDJ


In [10]:
cipher_text, _ = encrypt_Hill(message, Hill_matrix, mode=True)
print(cipher_text)

Wt pzoy ou Aibph Gkk Icb xcba xuydj 1362


In [11]:
# in ma trận cipher ra file excel 
df = pd.DataFrame(cipher_matrix)
df.to_excel('output/cipher_matrix.xlsx', index=False, header=False)

In [12]:
def mod_inverse(a, m):
    a = a % m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return 1

In [13]:
def get_Hill_matrix_inv(Hill_matrix):
    det = int(np.round(np.linalg.det(Hill_matrix)))
    det_inv = mod_inverse(det, 26)
    Hill_matrix_inv = np.round(det_inv * np.linalg.inv(Hill_matrix) * det).astype(int) % 26
    
    return Hill_matrix_inv

In [14]:
Hill_matrix_inv = get_Hill_matrix_inv(Hill_matrix)
print(Hill_matrix_inv)

[[12  7  0 19]
 [19 22  3 19]
 [25 20  9 22]
 [ 0 16 23 17]]


In [15]:
# in ma trận Hill_matrix_inv ra file excel
df = pd.DataFrame(Hill_matrix_inv)
df.to_excel('output/Hill_matrix_inv.xlsx', index=False, header=False)

In [16]:
def fix_format_decrypted_message(cipher_text, decrypted_message):
    list_chars = list(cipher_text)
    for i in range(len(list_chars)):
        if not list_chars[i].isalpha():
            decrypted_message = decrypted_message[:i] + list_chars[i] + decrypted_message[i:]
            list_chars[i] = '-'
        if list_chars[i].islower():
            decrypted_message = decrypted_message[:i] + decrypted_message[i].lower() + decrypted_message[i+1:]
            
    return decrypted_message

In [17]:
def decrypt_Hill(cipher_text, Hill_matrix, mode=False):
    cipher_text_root = cipher_text
    Hill_matrix_inv = get_Hill_matrix_inv(Hill_matrix)
    cipher_text = cipher_text.upper()
    for i in cipher_text:
        if i not in alphabet:
            cipher_text = cipher_text.replace(i, '')
    cipher_text = [alphabet.index(i) for i in cipher_text]
    cipher_text = np.array(cipher_text)
    cipher_text = cipher_text.reshape(-1, 4)
    decrypted_message = np.dot(cipher_text, Hill_matrix_inv) % 26
    decrypted_message = decrypted_message.flatten()
    decrypted_message = [alphabet[i] for i in decrypted_message]
    decrypted_message = ''.join(decrypted_message)
    if mode:
        decrypted_message = fix_format_decrypted_message(cipher_text_root, decrypted_message)
    return decrypted_message

In [18]:
decrypted_message = decrypt_Hill(cipher_text, Hill_matrix, mode=True)
print(decrypted_message)

My name is Vuong Van Duy from group 1362


In [19]:
# in toàn bộ dữ liệu ra file text. 
with open('output_cipher/data_HILL.txt', 'w') as f:
    f.write(f'Plaintext: {message}\n\n')
    f.write(f'Hill matrix:\n{Hill_matrix}\n\n')
    f.write(f'Plaintext matrix:\n{plaintext_matrix}\n\n')
    f.write(f'Ciphertext:\n{cipher_text}\n\n')
    f.write(f'Cipher matrix:\n{cipher_matrix}\n\n')
    f.write(f'Hill matrix inversed:\n{Hill_matrix_inv}\n\n')
    f.write(f'Decrypted message:\n{decrypted_message}')